<a href="https://colab.research.google.com/github/eraysozer20/ResearchTopics/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet pythae

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 12.2 MB/s eta 0:00:00


In [3]:
from torch.utils.data import Dataset
import torch
from tqdm.auto import tqdm
import torchvision.datasets as datasets

from matplotlib import pyplot as plt

import numpy as np

from pythae.pipelines import GenerationPipeline
from pythae.models import VAE, VAEConfig
from pythae.trainers import BaseTrainerConfig, BaseTrainer
from pythae.trainers.training_callbacks import TrainHistoryCallback, TrainingCallback, CallbackHandler
from pythae.data import BaseDataset

from time import time
from datetime import timedelta
init_time = time()

device = torch.device("cuda")

torch.__version__, torch.cuda.is_available(), device

('2.5.0+cu121', False, device(type='cuda'))

## VAE ResNet

In [5]:
from PIL import Image
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader, random_split
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms
from pythae.models import VAE, VAEConfig
from pythae.trainers import BaseTrainerConfig, BaseTrainer
from pythae.data import BaseDataset
from pythae.models.nn.benchmarks.mnist import Encoder_ResNet_VAE_MNIST, Decoder_ResNet_AE_MNIST
from pythae.trainers.training_callbacks import TrainingCallback


In [6]:
!git clone  https://github.com/eraysozer20/ResearchTopics.git

fatal: destination path 'ResearchTopics' already exists and is not an empty directory.


In [7]:
!python3 /content/ResearchTopics/amputation_step/amputator.py

2024-10-24 19:14:22.253690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 19:14:22.309723: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 19:14:22.327379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 19:14:22.375967: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 19:14:24.649748: W tensorflow/compiler/tf2

In [ ]:
# loading images class
class TransformedMNISTDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = self.image_filenames[idx]
        image = Image.open(img_path).convert('L')  # changing it from RgB to binary scale
        if self.transform:
            image = self.transform(image)
        return image


# dataset transformation
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Changing the size to 28x28
    transforms.ToTensor(),        # Changing to tensors
])


image_dir = '/content/transformed_mnist'
transformed_dataset = TransformedMNISTDataset(image_dir=image_dir, transform=transform)

# splitting the dataset
dataset_size = len(transformed_dataset)
train_size = int(0.8 * dataset_size)
eval_size = dataset_size - train_size

train_dataset, eval_dataset = random_split(transformed_dataset, [train_size, eval_size])

# dummy creation
dummy_train_labels = torch.zeros(train_size)
dummy_eval_labels = torch.zeros(eval_size)


train_data = BaseDataset(
    data=torch.cat([x.unsqueeze(0) for x in train_dataset], dim=0),
    labels=dummy_train_labels
)

eval_data = BaseDataset(
    data=torch.cat([x.unsqueeze(0) for x in eval_dataset], dim=0),
    labels=dummy_eval_labels
)

# VAE with ResNet Configuration
vae_resnet_config = VAEConfig(
    input_dim=(1, 28, 28),  # entrance size
    latent_dim=16           # latent space size
)

# VAE cowithn ResNet
vae_resnet_model = VAE(
    model_config=vae_resnet_config,
    encoder=Encoder_ResNet_VAE_MNIST(vae_resnet_config),  # ResNet encoder
    decoder=Decoder_ResNet_AE_MNIST(vae_resnet_config)    # ResNet decoder
)

# Callback for loss curves
class LossHistoryCallback(TrainingCallback):
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, training_config, logs, **kwargs):
        train_loss = logs.get("train_epoch_loss", None)
        eval_loss = logs.get("eval_epoch_loss", None)

        if train_loss is not None:
            self.train_losses.append(train_loss)

        if eval_loss is not None:
            self.eval_losses.append(eval_loss)


loss_callback = LossHistoryCallback()

# Trainer
trainer_resnet_config = BaseTrainerConfig(
    output_dir='my_resnet_model',
    num_epochs=100,
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    optimizer_cls="AdamW",           #  AdamW Optimizor
    optimizer_params={"weight_decay": 0.05}  # Regularization
)


trainer_resnet = BaseTrainer(
    model=vae_resnet_model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    training_config=trainer_resnet_config,
    callbacks=[loss_callback]
)

# training
trainer_resnet.train()



In [ ]:

plt.figure(figsize=(12,6))
plt.plot(loss_callback.train_losses, label='Training Loss', color='blue', linestyle='-')
plt.plot(loss_callback.eval_losses, label='Validation Loss', color='orange', linestyle='--')
plt.title("Loss curves in training (ResNet)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

output_image_dir = './output_images_resnet'
os.makedirs(output_image_dir, exist_ok=True)

output_latent_dir = './output_latents_resnet'
os.makedirs(output_latent_dir, exist_ok=True)

# evaluation
vae_resnet_model.eval()

# data loader for eval set
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)
all_latents = []

with torch.no_grad():
    for batch_idx, eval_samples in enumerate(eval_loader):

        reconstructions = vae_resnet_model.reconstruct(eval_samples.to(vae_resnet_model.device)).cpu()


        for i, (orig_image, recon_image) in enumerate(zip(eval_samples, reconstructions)):
            orig_img_path = os.path.join(output_image_dir, f'original_{batch_idx}_{i}.png')
            recon_img_path = os.path.join(output_image_dir, f'reconstructed_{batch_idx}_{i}.png')

            orig_img_pil = Image.fromarray((orig_image.squeeze().numpy() * 255).astype(np.uint8))
            recon_img_pil = Image.fromarray((recon_image.squeeze().numpy() * 255).astype(np.uint8))

            orig_img_pil.save(orig_img_path)
            recon_img_pil.save(recon_img_path)

        latents = vae_resnet_model.encoder(eval_samples.to(vae_resnet_model.device)).embedding.cpu().numpy()
        all_latents.append(latents)


latent_array = np.concatenate(all_latents, axis=0)
np.save(os.path.join(output_latent_dir, 'latents_resnet.npy'), latent_array)

In [ ]:
# Zip the directory
!zip -r output_images_resnet.zip ./output_images_resnet
!zip -r output_latents_resnet.zip ./output_latents_resnet

In [ ]:
from google.colab import files

# Download the zipped directories
files.download('output_images_resnet.zip')
files.download('output_latents_resnet.zip')

In [ ]:
# Visualizar algunas imágenes originales y sus reconstrucciones
vae_resnet_model.eval()  # Poner el modelo en modo de evaluación

# Obtener un lote de imágenes del dataset de evaluación
eval_samples = next(iter(DataLoader(eval_dataset, batch_size=10, shuffle=True)))

# Obtener las imágenes reconstruidas
with torch.no_grad():
    reconstructions = vae_resnet_model.reconstruct(eval_samples.to(vae_resnet_model.device)).cpu()

# Plotear imágenes originales y reconstruidas
fig, axes = plt.subplots(nrows=2, ncols=10, figsize=(15, 3))

# Mostrar las imágenes originales
for i in range(10):
    axes[0, i].imshow(eval_samples[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[0, i].set_title('Original')

# Mostrar las imágenes reconstruidas
for i in range(10):
    axes[1, i].imshow(reconstructions[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    axes[1, i].set_title('constructed')

plt.tight_layout()
plt.show()

## VAE MLP

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from pythae.models import VAE, VAEConfig
from pythae.trainers import BaseTrainerConfig, BaseTrainer
from pythae.data import BaseDataset
from pythae.trainers.training_callbacks import TrainingCallback

# Step 4: Define a dataset class to load the transformed MNIST images
class TransformedMNISTDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = self.image_filenames[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image

# Step 5: Define the transformations needed for the dataset
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize the images to 28x28
    transforms.ToTensor(),        # Convert images to tensors
])

# Load the dataset of transformed MNIST images
image_dir = '/content/transformed_mnist'  # Path to the folder with transformed images
transformed_dataset = TransformedMNISTDataset(image_dir=image_dir, transform=transform)

# Step 6: Split the dataset into training (80%) and evaluation (20%)
dataset_size = len(transformed_dataset)
train_size = int(0.8 * dataset_size)
eval_size = dataset_size - train_size

train_dataset, eval_dataset = random_split(transformed_dataset, [train_size, eval_size])

# Step 7: Create dummy labels for training and evaluation sets
dummy_train_labels = torch.zeros(train_size)
dummy_eval_labels = torch.zeros(eval_size)

# Convert the datasets to the format required by pythae
train_data = BaseDataset(
    data=torch.cat([x.unsqueeze(0) for x in train_dataset], dim=0),
    labels=dummy_train_labels
)

eval_data = BaseDataset(
    data=torch.cat([x.unsqueeze(0) for x in eval_dataset], dim=0),
    labels=dummy_eval_labels
)

# Step 8: Configure the VAE model (you can modify the latent dimension if needed)
vae_config = VAEConfig(
    input_dim=(1, 28, 28),  # Input shape matching the MNIST dataset
    latent_dim=16           # Latent space dimension
)

# Instantiate the VAE model
vae_model = VAE(model_config=vae_config)

# Step 9: Callback for logging loss during training
class LossHistoryCallback(TrainingCallback):
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, training_config, logs, **kwargs):
        train_loss = logs.get("train_epoch_loss", None)
        eval_loss = logs.get("eval_epoch_loss", None)

        if train_loss is not None:
            self.train_losses.append(train_loss)
        if eval_loss is not None:
            self.eval_losses.append(eval_loss)

# Step 10: Configure the trainer with a callback to log the losses
trainer_config = BaseTrainerConfig(
    output_dir='my_model',           # Directory to save the trained model
    num_epochs=100,                  # Number of epochs for training
    learning_rate=1e-4,              # Learning rate
    per_device_train_batch_size=64,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    optimizer_cls="AdamW",           # Optimizer AdamW
    optimizer_params={"weight_decay": 0.05}  # Regularization
)

# Instantiate the callback
loss_callback = LossHistoryCallback()

# Step 11: Instantiate the trainer and start training
trainer = BaseTrainer(
    model=vae_model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    training_config=trainer_config,
    callbacks=[loss_callback]  # Add custom callback to log losses
)

# Train the model
trainer.train()

# Step 12: Visualize the loss curves for training and evaluation
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(loss_callback.train_losses, label='Train Loss', color='blue', linestyle='-')
plt.plot(loss_callback.eval_losses, label='Eval Loss', color='orange', linestyle='--')
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# Step 13: Generate and save the output images and latent vectors
# Create directories for saving the generated images and latent vectors
output_image_dir = './output_images'
os.makedirs(output_image_dir, exist_ok=True)

output_latent_dir = './output_latents'
os.makedirs(output_latent_dir, exist_ok=True)

# Load the evaluation dataset into a DataLoader
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)

# Save the latent vectors and reconstructed images
all_latents = []

with torch.no_grad():
    for batch_idx, eval_samples in enumerate(eval_loader):
        reconstructions = vae_model.reconstruct(eval_samples.to(vae_model.device)).cpu()

        # Save original and reconstructed images
        for i, (orig_image, recon_image) in enumerate(zip(eval_samples, reconstructions)):
            orig_img_path = os.path.join(output_image_dir, f'original_{batch_idx}_{i}.png')
            recon_img_path = os.path.join(output_image_dir, f'reconstructed_{batch_idx}_{i}.png')

            orig_img_pil = Image.fromarray((orig_image.squeeze().numpy() * 255).astype(np.uint8))
            recon_img_pil = Image.fromarray((recon_image.squeeze().numpy() * 255).astype(np.uint8))

            orig_img_pil.save(orig_img_path)
            recon_img_pil.save(recon_img_path)

        # Save latent vectors
        latents = vae_model.encoder(eval_samples.to(vae_model.device)).embedding.cpu().numpy()
        all_latents.append(latents)

# Save the latent vectors as a .npy file
latent_array = np.concatenate(all_latents, axis=0)
np.save(os.path.join(output_latent_dir, 'latents.npy'), latent_array)

print("Images and latent vectors saved successfully.")

# Step 14: Zip the directories
!zip -r output_images.zip ./output_images
!zip -r output_latents.zip ./output_latents

# Step 15: Download the zipped directories
from google.colab import files
files.download('output_images.zip')
files.download('output_latents.zip')


Model passed sanity check !
Ready for training.

INFO:pythae.trainers.base_trainer.base_trainer:Model passed sanity check !
Ready for training.

Created my_model folder since did not exist.

INFO:pythae.trainers.base_trainer.base_trainer:Created my_model folder since did not exist.

Created my_model/VAE_training_2024-10-24_19-17-02. 
Training config, checkpoints and final model will be saved here.

INFO:pythae.trainers.base_trainer.base_trainer:Created my_model/VAE_training_2024-10-24_19-17-02. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 100
 - per_device_train_batch_size: 64
 - per_device_eval_batch_size: 64
 - checkpoint saving every: None
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.05
)
Scheduler: None

INFO:pythae.trainers.base_trainer.base_tr

Training of epoch 1/100:   0%|          | 0/750 [00:00<?, ?batch/s]

Eval of epoch 1/100:   0%|          | 0/188 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 17.9836
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 17.9836
Eval loss: 13.7466
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 13.7466
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 2/100:   0%|          | 0/750 [00:00<?, ?batch/s]

...

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 12.6826
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 12.6826
Eval loss: 12.6393
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 12.6393
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 92/100:   0%|          | 0/750 [00:00<?, ?batch/s]

Eval of epoch 92/100:   0%|          | 0/188 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 12.6775
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 12.6775
Eval loss: 12.6512
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 12.6512
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 93/100:   0%|          | 0/750 [00:00<?, ?batch/s]

In [ ]:
# Visualizar algunas imágenes originales y sus reconstrucciones
vae_model.eval()  # Poner el modelo en modo de evaluación

# Obtener un lote de imágenes del dataset de evaluación
eval_samples = next(iter(DataLoader(eval_dataset, batch_size=10, shuffle=True)))

# Obtener las imágenes reconstruidas
with torch.no_grad():
    reconstructions = vae_model.reconstruct(eval_samples.to(vae_model.device)).cpu()

# Plotear imágenes originales y reconstruidas
fig, axes = plt.subplots(nrows=2, ncols=10, figsize=(15, 3))

# Mostrar las imágenes originales
for i in range(10):
    axes[0, i].imshow(eval_samples[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[0, i].set_title('Original')

# Mostrar las imágenes reconstruidas
for i in range(10):
    axes[1, i].imshow(reconstructions[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    axes[1, i].set_title('constructed')

plt.tight_layout()
plt.show()